In [154]:
import pandas as pd
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
dfDeliveries = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\IPL_Ball_by_Ball_2008_2022.csv")
dfMatches = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\archive\IPL_Matches_2008_2022.csv")
dfDeliveries = dfDeliveries.merge(dfMatches, on='ID')
dfDeliveries['BowlingTeam'] = np.where(dfDeliveries['BattingTeam']==dfDeliveries['Team1'],
                                       dfDeliveries['Team2'],
                                       dfDeliveries['Team1'])



dfDeliveries['delivery'] = 1
dfDeliveries['Dot_Balls'] = (dfDeliveries['batsman_run'] == 0).astype(int)
dfDeliveries['Singles'] = (dfDeliveries['batsman_run'] == 1).astype(int)
dfDeliveries['Fours'] = (dfDeliveries['batsman_run'] == 4).astype(int)
dfDeliveries['Sixes'] = (dfDeliveries['batsman_run'] == 6).astype(int)


def overClassifier(val):
    if val>=0 and val<6:
        return 'Power Play'
    elif val>=6 and val<15:
        return 'Middle Overs'
    else:
        return 'Death Overs'
dfDeliveries['PhaseOfPlay'] = dfDeliveries['overs'].map(overClassifier)
dfDeliveries[['overs','PhaseOfPlay']].sort_values('overs').drop_duplicates().reset_index(drop=True)
dfDeathOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Death Overs']
dfPowerplayOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Power Play Overs']
dfMiddleOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Middle Overs']

df=dfDeliveries  
dfbatting = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\all_bat.csv")
dfbowling = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\all_bowl.csv")
dfvenue= pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\venue.csv")
dfplayervsplayer= pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\pvp_all.csv")

In [155]:

class Player:
    def __init__(self, name, player_type= None):
        self.name = name
        self.player_type = player_type
    def __str__(self):
        return f"Player Name: {self.name}, Type: {self.player_type}"

class Batsman(Player):
    instances = []
    def __init__(self, name, **kwargs):
        super().__init__(name, "batsman")
        
        self.active_ratio_death = kwargs.get('Active Ratio_death')
        self.striking_ratio_death = kwargs.get('Striking Ratio_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.true_avg_death = kwargs.get('True Avg_death')
        self.active_ratio_middle = kwargs.get('Active Ratio_middle')
        self.striking_ratio_middle = kwargs.get('Striking Ratio_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.true_avg_middle = kwargs.get('True Avg_middle')
        self.active_ratio_powerplay = kwargs.get('Active Ratio_powerplay')
        self.striking_ratio_powerplay = kwargs.get('Striking Ratio_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.true_avg_powerplay = kwargs.get('True Avg_powerplay')
        self.instances.append(self)
    def __str__(self):
        return f"Batsman: {self.name}, True Strike Rate (Death): {self.true_sr_death}"
    @classmethod
    def print_all_instances(cls):
        # Iterate over all instances and print them
        for instance in cls.instances:
            print(instance)



class Bowler(Player):
    instances = []
    def __init__(self, name, **kwargs):
        super().__init__(name, "bowler")
        self.team = None
        self.true_economy_death = kwargs.get('True Economy_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.dot_percentage_death = kwargs.get('dotpercentage_death')
        self.containment_death = kwargs.get('Containment_death')
        self.true_economy_middle = kwargs.get('True Economy_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.dot_percentage_middle = kwargs.get('dotpercentage_middle')
        self.containment_middle = kwargs.get('Containment_middle')
        self.true_economy_powerplay = kwargs.get('True Economy_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.dot_percentage_powerplay = kwargs.get('dotpercentage_powerplay')
        self.containment_powerplay = kwargs.get('Containment_powerplay')
        self.instances.append(self)
    def __str__(self):
        return f"Bowler: {self.name}, Containment (Death): {self.containment_death}"
    @classmethod
    def print_all_instances(cls):
        # Iterate over all instances and print them
        for instance in cls.instances:
            print(instance)


class Venue:
    instances = []
    def __init__(self, venue_name, matches, total_run, dot_balls, is_wicket_delivery, delivery, fours, sixes, average_score, dot_ball_percentage, average_wickets_fallen, boundary_frequency):
        self.venue_name = venue_name
        self.matches = matches
        self.total_run = total_run
        self.dot_balls = dot_balls
        self.is_wicket_delivery = is_wicket_delivery
        self.delivery = delivery
        self.fours = fours
        self.sixes = sixes
        self.average_score = average_score
        self.dot_ball_percentage = dot_ball_percentage
        self.average_wickets_fallen = average_wickets_fallen
        self.boundary_frequency = boundary_frequency
        self.instances.append(self)
    @classmethod
    def print_all_instances(cls):
        # Iterate over all instances and print them
        for instance in cls.instances:
            print(instance)

def get_player_all_profiles(playing_xi_team1, dfbatting, dfbowling):
    # Initialize empty dicts to hold player profiles
    player_profiles = {}
    
    # Combine both teams into a single list to streamline processing
    all_players = set(playing_xi_team1 )
    
    # Search and Extract Profiles for Both Batting and Bowling
    for player in all_players:
        player_profiles[player] = {
            'batting': None,
            'bowling': None
        }
        
        # Check batting stats
        if player in dfbatting['batter'].values:
            player_profiles[player]['batting'] = dfbatting[dfbatting['batter'] == player].to_dict('records')[0]
        
        # Check bowling stats
        if player in dfbowling['bowler'].values:
            player_profiles[player]['bowling'] = dfbowling[dfbowling['bowler'] == player].to_dict('records')[0]
        
    return player_profiles


def get_venue_features(dfvenue, venue_name):
    # Filter the dataframe for the given venue_name
    venue_data = dfvenue[dfvenue['venue_name'] == venue_name].iloc[0]
    
    # Extract features from the filtered dataframe row
    venue = Venue(
        venue_name=venue_data['venue_name'],
        matches=venue_data['Matches'],
        total_run=venue_data['total_run'],
        dot_balls=venue_data['Dot_Balls'],
        is_wicket_delivery=venue_data['isWicketDelivery'],
        delivery=venue_data['delivery'],
        fours=venue_data['Fours'],
        sixes=venue_data['Sixes'],
        average_score=venue_data['Average Score'],
        dot_ball_percentage=venue_data['Dot Ball Percentage'],
        average_wickets_fallen=venue_data['Average Wickets Fallen'],
        boundary_frequency=venue_data['Boundary Frequency']
    )
    
    return venue





In [184]:
import random
def find_batsman_by_name(name):
    for batsman in batsmen:
        if batsman.name == name:
            return batsman
    return None
def find_bowler_by_name(name):
    for bowler in bowlers:
        if bowler.name == name:
            return bowler
    return None

def select_bowler(available_bowlers, previous_bowler, bowling_team):
    filtered_bowlers = [bowler for bowler in available_bowlers if bowler != previous_bowler and bowler.name in bowling_team]
    return random.choice(filtered_bowlers)



def simulate_match(team1, team2, venue):
    total_overs = 20
    innings_scores = []

    for inning_number in range(2):  # Two innings, one for each team
        if inning_number == 0:
            batsmen_instances = [find_batsman_by_name(batsman_name) for batsman_name in team1]
            bowler_instances = [find_bowler_by_name(bowler_name) for bowler_name in team2]
            batting_team = team1
            bowling_team = team2
        else:
            batsmen_instances = [find_batsman_by_name(batsman_name) for batsman_name in team2]
            bowler_instances = [find_bowler_by_name(bowler_name) for bowler_name in team1]
            batting_team = team2
            bowling_team = team1

        score = 0
        wickets = 0
        overs = 0
        current_batsmen_indices = [0, 1]  # Start with the first two batsmen
        next_batsman_index = 2  # Index for the next batsman
        previous_bowler = None  # To ensure the same bowler does not bowl consecutive overs

        current_batsmen = [batsmen_instances[current_batsmen_indices[0]],
                           batsmen_instances[current_batsmen_indices[1]]]
        
        current_batsman_index = 0  # Index indicating the current batsman facing the ball
        non_striker_index = 1 if current_batsman_index == 0 else 0  # Index indicating the non-striker
        previous_bowler = None

        while overs < total_overs and wickets < 10:
            # Update the previous bowler
            current_bowler = select_bowler(bowler_instances, previous_bowler,bowling_team)

            for ball in range(6):
                if wickets >= 10:
                    break  # All out before 20 overs
                
                outcome = simulate_ball(current_batsmen[current_batsman_index], current_bowler, venue, overs,
                                         current_batsmen[current_batsman_index].name,
                                         current_batsmen[non_striker_index].name, current_bowler.name,
                                         batting_team, bowling_team)

                if outcome != 'W':  # Runs scored
                    score += outcome
                    # Change strike after 1 run
                    if outcome % 2 == 1:
                        current_batsman_index, non_striker_index = non_striker_index, current_batsman_index
                else:  # Wicket falls
                    wickets += 1
                    if next_batsman_index < len(batting_team):
                        current_batsmen[current_batsman_index] = find_batsman_by_name(batting_team[next_batsman_index])
                        next_batsman_index += 1
                    else:
                        break  # No more batsmen left
                    
                    # New batsman takes strike
                    current_batsman_index = non_striker_index
                    non_striker_index = (current_batsman_index + 1) % 2
                
                

            overs += 1 
            print("Batting Team:", batting_team)
            print("Bowling Team:", bowling_team)# Increment overs after each set of 6 balls
            previous_bowler = current_bowler 
            # Change strike at the end of the over
            current_batsman_index, non_striker_index = non_striker_index, current_batsman_index

        innings_scores.append((score, wickets, overs))

    print_innings_scores(innings_scores)
    determine_winner(innings_scores)





def match_phase(overs):
    if overs <= 6:
        return 'powerplay'
    elif overs <= 14:
        return 'middle'
    else:
        return 'death'
def adjust_for_phase(player, phase):
    """
    Adjust player stats based on the match phase and player type (batsman or bowler).
    """
    if player.player_type == "batsman":
        if phase == 'powerplay':
            player_stats = (player.true_sr_powerplay, player.active_ratio_powerplay, player.striking_ratio_powerplay)
        elif phase == 'middle':
            player_stats = (player.true_sr_middle, player.active_ratio_middle, player.striking_ratio_middle)
        else:  # death
            player_stats = (player.true_sr_death, player.active_ratio_death, player.striking_ratio_death)
    elif player.player_type == "bowler":
        if phase == 'powerplay':
            player_stats = (player.true_economy_powerplay, player.true_sr_powerplay, player.dot_percentage_powerplay, player.containment_powerplay)
        elif phase == 'middle':
            player_stats = (player.true_economy_middle, player.true_sr_middle, player.dot_percentage_middle, player.containment_middle)
        else:  # death
            player_stats = (player.true_economy_death, player.true_sr_death, player.dot_percentage_death, player.containment_death)
    else:
        player_stats = ()

    return player_stats


def adjust_venue_stats(venue):
    """
    Prepare venue stats for use in simulation calculations.
    """
    return (venue.dot_ball_percentage, venue.boundary_frequency)

def simulate_ball(batsman, bowler, venue, overs, batsman_name, non_striker_name, bowler_name, batsman_team, bowler_team):
    # This example slightly reduces wicket probability and increases run scoring chances
    phase = match_phase(overs)
    batsman_stats = adjust_for_phase(batsman, phase)
    bowler_stats = adjust_for_phase(bowler, phase)
    
    # Example simplification for demonstration
    batsman_power = sum(batsman_stats) / len(batsman_stats)
    bowler_power = sum(bowler_stats) / len(bowler_stats)
    match_factor = batsman_power - bowler_power

    # Simulate the outcome
    outcome_probability = random.uniform(0, 100)
    if outcome_probability <= 70:  # Adjusted for more runs
        runs = random.choices([1, 2, 3, 4, 6], weights=[25, 20, 15, 25, 15], k=1)[0]
        print(f"Over {int(overs) + 1}.{int(((overs % 1) * 6) + 1)}: {batsman_name} scored {runs} runs against {bowler_name}, {non_striker_name} at the other end.")
        return runs
    elif outcome_probability <= 90:  # Less chance for a dot ball
        print(f"Over {int(overs) + 1}.{int(((overs % 1) * 6) + 1)}: {batsman_name} played a dot ball against {bowler_name}, {non_striker_name} at the other end.")
        return 0
    else:  # Reduced probability for wicket
        print(f"Over {int(overs) + 1}.{int(((overs % 1) * 6) + 1)}: {batsman_name} got out by {bowler_name}, {non_striker_name} at the non-striker's end.")
        return 'W'
    return 0

 # Default return to handle unexpected cases, ensuring function doesn't return None
 # Example: dot ball percentage to boundary frequency ratio



def print_innings_scores(innings_scores):
    for i, (score, wickets, overs) in enumerate(innings_scores, start=1):
        print(f"Innings {i}: Scored {score} for {wickets} wickets in {overs} overs.")

def determine_winner(innings_scores):
    team1_score, team2_score = innings_scores[0][0], innings_scores[1][0]
    if team1_score > team2_score:
        print(f"Team 1 wins by {team1_score - team2_score} runs.")
    elif team2_score > team1_score:
        # Calculate the wickets left for the winning team
        wickets_left = 10 - innings_scores[1][1]
        print(f"Team 2 wins by {wickets_left} wickets.")
    else:
        print("The match is a tie.")

In [185]:
team1 = ["A Symonds", "AC Gilchrist","SR Watson","AD Russell","BA Stokes","CH Gayle","CH Morris","JC Archer","KA Pollard","KP Pietersen","Shakib Al Hasan"] 
team2 = ["SR Tendulkar", "Z Khan","RG Sharma","V Kohli","MS Dhoni","KH Pandya","HH Pandya","JJ Bumrah","SK Raina","YK Pathan","YS Chahal"]
player_stats1 = get_player_all_profiles(team1,  dfbatting, dfbowling)

batsmen1 = []
bowlers1 = []

for player_name, stats in player_stats1.items():
    if 'batting' in stats and stats['batting'] is not None:
        batsmen1.append(Batsman(player_name, **stats['batting']))
    if 'bowling' in stats and stats['bowling'] is not None:
        bowlers1.append(Bowler(player_name, **stats['bowling']))
player_stats2 = get_player_all_profiles(team2,  dfbatting, dfbowling)

batsmen2 = []
bowlers2 = []

for player_name, stats in player_stats2.items():
    if 'batting' in stats and stats['batting'] is not None:
        batsmen2.append(Batsman(player_name, **stats['batting']))
    if 'bowling' in stats and stats['bowling'] is not None:
        bowlers2.append(Bowler(player_name, **stats['bowling']))

venue_name = "Brabourne Stadium"
venue1 = get_venue_features(dfvenue, venue_name)
simulate_match(team1,team2,venue1)

Over 1.1: A Symonds scored 2 runs against MS Dhoni, AC Gilchrist at the other end.
Over 1.1: A Symonds got out by MS Dhoni, AC Gilchrist at the non-striker's end.
Over 1.1: AC Gilchrist scored 1 runs against MS Dhoni, SR Watson at the other end.
Over 1.1: SR Watson scored 1 runs against MS Dhoni, AC Gilchrist at the other end.
Over 1.1: AC Gilchrist played a dot ball against MS Dhoni, SR Watson at the other end.
Over 1.1: AC Gilchrist scored 1 runs against MS Dhoni, SR Watson at the other end.
Batting Team: ['A Symonds', 'AC Gilchrist', 'SR Watson', 'AD Russell', 'BA Stokes', 'CH Gayle', 'CH Morris', 'JC Archer', 'KA Pollard', 'KP Pietersen', 'Shakib Al Hasan']
Bowling Team: ['SR Tendulkar', 'Z Khan', 'RG Sharma', 'V Kohli', 'MS Dhoni', 'KH Pandya', 'HH Pandya', 'JJ Bumrah', 'SK Raina', 'YK Pathan', 'YS Chahal']
Over 2.1: AC Gilchrist scored 2 runs against JJ Bumrah, SR Watson at the other end.
Over 2.1: AC Gilchrist played a dot ball against JJ Bumrah, SR Watson at the other end.
Over

In [ ]:
Bowler.print_all_instances()